In [2]:
from pathlib import Path
import matplotlib.pyplot as plt
import torch
from diffusers import StableDiffusionPipeline
from fastcore.all import concat
from huggingface_hub import notebook_login
from PIL import Image

In [3]:
torch.manual_seed(1)
if not (Path.home()/'.huggingface'/'token').exists(): notebook_login()

In [4]:
torch.cuda.empty_cache()

In [8]:
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4",revision="fp16",torch_dtype=torch.float32).to("cuda")

In [ ]:
pipe.enable_attention_slicing()

In [ ]:
prompt = "alien as an astronaut riding a horse"

In [ ]:
pipe(prompt).images[0]

In [ ]:
pipe(prompt, num_inference_steps=3).images[0] # you can pass the number of inference steps as the input

In [ ]:
pipe(prompt, num_inference_steps=16).images[0]

In [ ]:
num_rows,num_cols = 4,4
prompts = [prompt]*num_cols

In [ ]:
#to show images in a grid
def image_grid(imgs, rows, cols):
    w,h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    for i, img in enumerate(imgs): grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
images = concat(pipe(prompts, guidance_scale=g).images for g in [1.1,3,7,14]) #guidance scale how much we should focus on the caption guidance scale of 14 has overshooted ig
#concat is a function imported above

In [ ]:
image_grid(images,rows=num_rows,cols=num_cols)

In [ ]:
# negative prompts

In [ ]:
#normal
prompt = "Labrador in the style of Vermeer"
pipe(prompt).images[0]


In [ ]:
pipe(prompt, negative_prompt="blue").images[0]#negative prompt of blue removes blue color from the image

### Image to Image

Even though Stable Diffusion was trained to generate images, and optionally drive the generation using text conditioning, we can use the raw image diffusion process for other tasks.

For example, instead of starting from pure noise, we can start from an image an add a certain amount of noise to it. We are replacing the initial steps of the denoising and pretending our image is what the algorithm came up with. Then we continue the diffusion process from that state as usual.

This usually preserves the composition although details may change a lot. It's great for sketches!

These operations (provide an initial image, add some noise to it and run diffusion from there) can be automatically performed by a special image to image pipeline: `StableDiffusionImg2ImgPipeline`. This is the source code for its [`__call__` method](https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion_img2img.py#L124).

In [11]:
# __call__ enables objects of a class to be called as a function
from diffusers import StableDiffusionImg2ImgPipeline
from fastdownload import FastDownload

In [ ]:
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    revision="fp16",
    torch_dtype=torch.float16,
).to("cuda") 
#same as above

In [ ]:
p = FastDownload().download('https://s3.amazonaws.com/moonup/production/uploads/1664665907257-noauth.png')
init_image = Image.open(p).convert("RGB")
init_image#rather than starting the diffusion with noise 

In [ ]:
torch.manual_seed(1000)
prompt = "Wolf howling at the moon, photorealistic 4K"
images = pipe(prompt=prompt, num_images_per_prompt=3, init_image=init_image, strength=0.8, num_inference_steps=50).images
image_grid(images, rows=1, cols=3)

In [ ]:
# this was image to image